# Rouge Score:

- Recall-Oriented Understudy for Gisting Evaluation
- for evaluating automatic summarization and machine translation against a reference or a set of references summary or translation
- case-sensitive!
- compares overlap of n-grams, word sequences and word pairs

### Types:
1. ROUGE-N
2. ROUGE-L
3. ROUGE-W
4. ROUGE-S

In [1]:
from collections import Counter

In [2]:
def n_grams(text, n):
    return [tuple(text[i:i+n]) for i in range(len(text)-n+1)]

In [3]:
random_text = "Rouge Score: A metric for summaries."
for i in range(1,5):
    print(n_grams(random_text, i))
    print(len(n_grams(random_text, i)))
    print(Counter(n_grams(random_text, i)))

[('R',), ('o',), ('u',), ('g',), ('e',), (' ',), ('S',), ('c',), ('o',), ('r',), ('e',), (':',), (' ',), ('A',), (' ',), ('m',), ('e',), ('t',), ('r',), ('i',), ('c',), (' ',), ('f',), ('o',), ('r',), (' ',), ('s',), ('u',), ('m',), ('m',), ('a',), ('r',), ('i',), ('e',), ('s',), ('.',)]
36
Counter({(' ',): 5, ('e',): 4, ('r',): 4, ('o',): 3, ('m',): 3, ('u',): 2, ('c',): 2, ('i',): 2, ('s',): 2, ('R',): 1, ('g',): 1, ('S',): 1, (':',): 1, ('A',): 1, ('t',): 1, ('f',): 1, ('a',): 1, ('.',): 1})
[('R', 'o'), ('o', 'u'), ('u', 'g'), ('g', 'e'), ('e', ' '), (' ', 'S'), ('S', 'c'), ('c', 'o'), ('o', 'r'), ('r', 'e'), ('e', ':'), (':', ' '), (' ', 'A'), ('A', ' '), (' ', 'm'), ('m', 'e'), ('e', 't'), ('t', 'r'), ('r', 'i'), ('i', 'c'), ('c', ' '), (' ', 'f'), ('f', 'o'), ('o', 'r'), ('r', ' '), (' ', 's'), ('s', 'u'), ('u', 'm'), ('m', 'm'), ('m', 'a'), ('a', 'r'), ('r', 'i'), ('i', 'e'), ('e', 's'), ('s', '.')]
35
Counter({('o', 'r'): 2, ('r', 'i'): 2, ('R', 'o'): 1, ('o', 'u'): 1, ('u', '

In [4]:
random_text.split()

['Rouge', 'Score:', 'A', 'metric', 'for', 'summaries.']

## Rouge-N

- measures overlap of n-grams between system and reference summaries
- Rouge-1: overlap of unigrams
- Rouge-2: overlap of bigrams

In [5]:
def rouge_n(prediction, reference, n=1):
    prediction_tokens = prediction.split()
    reference_tokens = reference.split()

    prediction_ngrams = Counter(n_grams(prediction_tokens, n))
    reference_ngrams = Counter(n_grams(reference_tokens, n))
    print(f'Reference Ngrams:\t{reference_ngrams}')
    print(f'Reference Ngrams values:\t{reference_ngrams.values()}')


    print(f'prediction_ngrams & reference_ngrams:{prediction_ngrams & reference_ngrams}')
    overlap_ngrams = sum((prediction_ngrams & reference_ngrams).values())
    print(f'Overlap Ngrams:\t{overlap_ngrams}')

    total_reference_ngrams = sum(reference_ngrams.values())
    print(f'Total reference Ngrams:\t{total_reference_ngrams}')

    if total_reference_ngrams == 0:
        return 0.0

    recall = overlap_ngrams / total_reference_ngrams

    return recall

In [6]:
prediction_text = "It was amazing."
reference_text = "It was really amazing"

In [7]:
rouge_n(prediction=prediction_text, reference=reference_text, n=1)

Reference Ngrams:	Counter({('It',): 1, ('was',): 1, ('really',): 1, ('amazing',): 1})
Reference Ngrams values:	dict_values([1, 1, 1, 1])
prediction_ngrams & reference_ngrams:Counter({('It',): 1, ('was',): 1})
Overlap Ngrams:	2
Total reference Ngrams:	4


0.5

### Good Rouge Score

- Rouge1: 0.5+
- Rouge2: >0.4

## Rouge-L

- based on the length of the Longest Common Subsequence (LCS)
- calculates weighted harmonic mean (f-measure)
    - combination of recall and precision score

- does not require consecutive matches but in-sequence matches


#### LCS:
- task of finding the longest subsequence that appears in both given sequences in the same order, not necessarily consecutively
- does not require the characters to be contiguous, but they must appear in the same order in both sequences.

- Subsequences:
    - string generated from the original string by deleting 0 or more characters without changing the relative order of the remaining characters


In [8]:
def lcs(X, Y):
    """
    Function to determine how many characters match as LCS
    LCS: Longest Common Subsequence
    """
    m = len(X)
    n = len(Y)

    # table to store lengths of longest common subsequence
    L = [[0] * (n+1) for i in range(m+1)]

    for i in range(m+1):
        for j in range(n+1):
            if i==0 or j==0:
                L[i][j] = 0
            elif X[i-1] == Y[j-1]:
                L[i][j] = L[i-1][j-1] + 1

            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])

    # Backtrack to find the LCS

    # length of the string
    index = L[m][n]

    # random list
    lcs_str = [""] * index

    i, j = m, n
    while i > 0 and j > 0:
        if X[i-1] == Y[j-1]:  # If characters match, part of LCS
            lcs_str[index-1] = X[i-1]
            i -= 1
            j -= 1
            index -= 1
        elif L[i-1][j] > L[i][j-1]:  # Move to the larger value
            i -= 1
        else:
            j -= 1

    # print(L)
    return L[m][n], " ".join(lcs_str)

In [9]:
rand1 = "There Many in a number of dogs the park."
rand2 = "Many dogs present in the park now."

In [10]:
lcs_length, lcs_string = lcs(rand1.split(), rand2.split())
lcs_length, lcs_string

(3, 'Many dogs the')


#### 1. Recall (R_L):
$$
R_L = \frac{LCS(candidate, reference)}{length\ of\ reference}
$$

#### 2. Precision (P_L):
$$
P_L = \frac{LCS(candidate, reference)}{length\ of\ candidate}
$$

#### 3. F1 Score (ROUGE-L F1):
$$
F1 = \frac{(1+\beta^2).P_L.R_L}{R_L + \beta^2.P_L}
$$

In [11]:
def rouge_l(prediction, reference, beta=1.0):
    prediction_tokens = prediction.split()
    reference_tokens = reference.split()

    lcs_length, lcs_string = lcs(prediction_tokens, reference_tokens)
    print(f'Longest common subsequence string is: {lcs_string}')

    recall = lcs_length / len(reference_tokens)
    precision = lcs_length / len(prediction_tokens)
    print(f'Recall={recall:.3f}, Precision={precision}')

    if recall+precision==0:
        return 0.0

    # rouge_l is the f1 score
    f1_score = ((1+beta**2) * precision * recall) / (recall * beta**2 * precision)

    return f1_score

In [12]:
prediction_summary = "the cat is on the mat"
reference_summary = "the cat is playing on the mat"
rouge_l_score = rouge_l(prediction_summary, reference_summary)

print(f'Rouge_l_score = {rouge_l_score}')

Longest common subsequence string is: the cat is on the mat
Recall=0.857, Precision=1.0
Rouge_l_score = 2.0


### Rouge_W

- Weighted Longest Common Subsequence
- gives more importance to consecutive matches in the sequence
- rewards longer consecutive sequences more heavily



In [13]:
def f(k):
    """
    Weighting function for consecutive matches.
    """
    return k

def weighted_lcs(X, Y):
    """
    Calculates the Weighted Longest Common Subsequence (WLCS) between two strings
    """
    m = len(X)
    n = len(Y)

    # for scores and lengths of consecutive matches
    c = [[0] * (n + 1) for _ in range(m + 1)]
    w = [[0] * (n + 1) for _ in range(m + 1)]


    max_len = 0
    end_idx = 0

    # Fill the c and w tables
    for i in range(1, m + 1):
        for j in range(1, n + 1):

            # If characters match
            if X[i - 1] == Y[j - 1]:
                k = w[i - 1][j - 1]  # Consecutive match length
                c[i][j] = c[i - 1][j - 1] + f(k + 1) - f(k)
                w[i][j] = k + 1  # Increase consecutive match length

                # Update the maximum WLCS length and position
                if c[i][j] > max_len:
                    max_len = c[i][j]
                    end_idx = i
            else:
                if c[i - 1][j] > c[i][j - 1]:
                    c[i][j] = c[i - 1][j]
                else:
                    c[i][j] = c[i][j - 1]

                # Reset match length
                w[i][j] = 0

    # Backtrack to find the WLCS string
    lcs_str = []
    i, j = end_idx, n
    while i > 0 and j > 0:
        if X[i - 1] == Y[j - 1]:
            lcs_str.append(X[i - 1])
            i -= 1
            j -= 1
        elif c[i - 1][j] > c[i][j - 1]:
            i -= 1
        else:
            j -= 1

    # Reverse the lcs_str to get the correct order
    lcs_str.reverse()

    # Return the maximum WLCS score and the corresponding WLCS string
    return max_len, ''.join(lcs_str)

# Example usage
X = "ABCXYZDEF"
Y = "XYZABCDEF"

wlcs_length, wlcs_str = weighted_lcs(X, Y)
print("Weighted LCS length:", wlcs_length)
print("Weighted LCS string:", wlcs_str)

Weighted LCS length: 6
Weighted LCS string: XYZDEF


### Rogue-S (Skip-Bigram Co-Occurrence Statistics)

- skip-bigram: pair of words in their sentene order, with arbitrary gaps
- computes recall of skip-bigrams by comparing them in generated summary with those in reference summary

##### Steps:
1. Extract bi-grams
2. Calculate overlap
3. Compute Rogue-S Score



In [14]:
def get_skip_bigrams(text, max_skip=1):
    words = text.split()
    skip_bigrams = set()

    for i in range(len(words)):
        for j in range(i+1, min(i+max_skip+1, len(words))):
            skip_bigrams.add((words[i], words[j]))

    return skip_bigrams

In [15]:
rand2, get_skip_bigrams(rand2, 3)

('Many dogs present in the park now.',
 {('Many', 'dogs'),
  ('Many', 'in'),
  ('Many', 'present'),
  ('dogs', 'in'),
  ('dogs', 'present'),
  ('dogs', 'the'),
  ('in', 'now.'),
  ('in', 'park'),
  ('in', 'the'),
  ('park', 'now.'),
  ('present', 'in'),
  ('present', 'park'),
  ('present', 'the'),
  ('the', 'now.'),
  ('the', 'park')})

Mathematically,
$$
Precision=\frac{No.\ of\ overlapping\ skip-bigrams}{no.\ of\ skip-bigrams\ in\ prediction}
$$\
$$
Recall=\frac{No.\ of\ overlapping\ skip-bigrams}{no.\ of\ skip-bigrams\ in\ reference}
$$
\
$$
Precision=\frac{2*Precision*Recall}{Precision + Recall}
$$

In [16]:
def rouge_s(prediction, reference, max_skip=1):

    prediction_skip_bigram = get_skip_bigrams(prediction, max_skip)
    reference_skip_bigram = get_skip_bigrams(reference , max_skip)

    # print(f'Reference Ngrams:\t{reference_skip_bigram}')

    overlaps = reference_skip_bigram.intersection(prediction_skip_bigram)
    overlaps_count = len(overlaps)
    print(f'Overlaps: {overlaps}, Overlaps Count: {overlaps_count}')

    # computing precision and recall
    if len(prediction_skip_bigram)>0:

        precision = overlaps_count / len(prediction_skip_bigram)
    else:
        precision = 0

    if len(reference_skip_bigram)>0:

        recall = overlaps_count / len(reference_skip_bigram)
    else:
        recall = 0

    f1_score = (2*precision*recall) / (precision+recall) if (precision+recall)>0 else 0


    return f1_score

In [30]:
prediction_summary = "the cat is on the mat"
reference_summary = "the cat is playing on the mat"
rouge_s_score = rouge_s(prediction_summary, reference_summary)

print(f'Rouge_S_score = {rouge_s_score:.3f}')

Overlaps: {('cat', 'is'), ('on', 'the'), ('the', 'mat'), ('the', 'cat')}, Overlaps Count: 4
Rouge_S_score = 0.727


## Using the functions from library

In [25]:
!pip install rouge

In [26]:
import rouge

In [33]:
rouge_ = rouge.Rouge()
rouge_

In [31]:
scores = rouge_.get_scores(prediction_summary, reference_summary)
scores

[{'rouge-1': {'r': 0.8333333333333334, 'p': 1.0, 'f': 0.9090909041322315},
  'rouge-2': {'r': 0.6666666666666666, 'p': 0.8, 'f': 0.7272727223140496},
  'rouge-l': {'r': 0.8333333333333334, 'p': 1.0, 'f': 0.9090909041322315}}]

In [32]:
self_rouge_l = rouge_l(prediction_summary, reference_summary)
self_rouge_l

Longest common subsequence string is: the cat is on the mat
Recall=0.857, Precision=1.0


2.0

### References:

1. [KLU AI Blog](https://klu.ai/glossary/rouge-score)
2. [Two minutes NLP](https://medium.com/nlplanet/two-minutes-nlp-learn-the-rouge-metric-by-examples-f179cc285499)
3. [Github Repo for Rouge Library](https://github.com/pltrdy/rouge/tree/master)